# Skytrax Web Scrape for all
* 改成可以指定時間
* 要爬取所有資料

# Install & Imports

In [34]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import math

In [95]:
# settings
pd.set_option('precision', 0)

# Try

In [2]:
# 'https://www.airlinequality.com/airline-reviews/azur-air-germany?sortby=post_date%3ADesc&pagesize=100'
# tmp = review_url_list[0]
tmp = 'https://www.airlinequality.com/airline-reviews/ab-aviation/?sortby=post_date%3ADesc&pagesize=100'

import re
regex = re.compile(r'airline-reviews/(\S*)\?sortby=post_date%3ADesc&pagesize=100')
match = regex.search(tmp)
print(match.group(1))

ab-aviation/


In [1]:
# len(review_url_list)
# 547

In [60]:
# use beautiful soup to parse reviews
# 直接找呈現100則評論的頁面
# 內容網址 : 'https://www.airlinequality.com/airline-reviews/ab-aviation/?sortby=post_date%3ADesc&pagesize=100'
url = 'https://www.airlinequality.com/airline-reviews/air-france/page/1/?sortby=post_date%3ADesc&pagesize=100'
# for url in review_url_list[:1]:
print(url, '\n')
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

allarticles = soup.findAll('article')

articles = allarticles[0].findAll('article')
print('Number of reviews: ', len(articles))


if len(articles)>0:    
    for idx, one in enumerate(articles):    
        print('\nReview No.', idx+1)
        #print(one) # If we want to look at the complete section source

        print('by:',one.find('span',{'itemprop': 'name'}).text)
        print('Published:',one.find('time',{'itemprop': 'datePublished'}).text)
        print('Ratings:',one.find('span',{'itemprop': 'ratingValue'}).text + '/' +one.find('span',{'itemprop': 'bestRating'}).text)
        print('Header:',one.find('h2',{'class': 'text_header'}).text )
        print(one.find('div',{'class': 'text_content'}).text.split('|')[1])


        data_cnt =0
        star_cnt =0

        for element in one.find_all('tr'):
            print(element.find_all('td')[0]['class'][1])

            if not (element.find_all('td')[1].text).isnumeric():
                print('the text : ' , element.find_all('td')[1].text)
                data_cnt+=1
            else:
                stars = len(element.find_all('span',{'class': 'star fill'}))
                star_cnt+=1
                print(stars)

            # if stars == 0:
            #     rating_value = (element.find_all('td')[1]).text
            #     record[rating_element] = rating_value.strip()
            # else:
            #     record[rating_element] = int(stars)

        print(data_cnt , star_cnt)



https://www.airlinequality.com/airline-reviews/air-france/page/1/?sortby=post_date%3ADesc&pagesize=100 

Number of reviews:  100

Review No. 1
by: A Heale
Published: 10th January 2023
Ratings: 9/10
Header: "customer service is outstanding"
  Flown with Air France many times over the last 20-30 years. This flight was just a highlight about why we use Air France whenever possible. Cabin crew are very friendly, happy to have a conversation and make you feel very safe and comfortable on the flight. Short haul food is very good, and a wide range of free drinks including wine available to economy passengers is great. Luggage policies are very reasonable and it doesn’t cost much to add on extra bags or suitcases. Boarding was very efficient and both our flights were on-time. Highly recommend using AF, customer service is outstanding.
aircraft
the text :  A320
type_of_traveller
the text :  Family Leisure
cabin_flown
the text :  Economy Class
route
the text :  Edinburgh to Paris
date_flown
the 

5
food_and_beverages
5
inflight_entertainment
2
ground_service
5
wifi_and_connectivity
5
value_for_money
5
recommended
the text :  yes
6 7

Review No. 82
by: N Hugin
Published: 15th June 2022
Ratings: 10/10
Header: "Was pleasantly surprised with Air France"
  Was pleasantly surprised with Air France as I was expecting the worst experience. The aircraft felt up to date, entertainment system worked fine and had plenty of choices. Crew were very friendly and personally greeted us and asked if we needed anything. The meal services were great, the main meal was the best one I had in economy for quite sometime as mostly they are inedible but this was good, chicken with mushrooms and polenta, a small salad with Camargue rice and a delicious small pastry for dessert, coffee and wine. The meal service before landing was presented in a small gift bag and contents were tasty, a nice touch I thought.
aircraft
the text :  A330
type_of_traveller
the text :  Couple Leisure
cabin_flown
the text :  Eco

# Functions

In [62]:
def get_airlines() -> list:
    
    '''
    get all the airlines and set the initial url to the first page of latest reviews
    '''
    
    
    # 各航空的 url
    url = 'https://www.airlinequality.com/review-pages/a-z-airline-reviews/'
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    review_url_list =[]

    # tab content
    tabs = soup.findAll('div',{'class' :'tabs-content'})
    tab = tabs[0].findAll('div', {'class' : 'content'})

    str_ = 'https://www.airlinequality.com/airline-reviews/'



    if len(tab)>0:    
        for idx, one in enumerate(tab):    
    #         print('tab No.', idx+1)
            for a in one.find_all('a', href=True):
                regex = re.compile(r'airline-reviews/(\S*)"')
                match = regex.search(str(a))
                url = str_+match.group(1)+'/page/1/?sortby=post_date%3ADesc&pagesize=100'
                review_url_list.append(url)
                
    return review_url_list


In [63]:
def get_pages( url : str ) -> int:
    
    # find how many page
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    divs = soup.findAll('div')

    # 有其他分頁
    if divs[0].find('div',{'class': 'pagination-total'}):
        div = divs[0].find('div',{'class': 'pagination-total'}).text
        #1 to 100 of 1218 Reviews
        num_reviews = re.compile(r'1 to 100 of (\d*) Reviews') 
        match = num_reviews.search(div)
        return math.ceil(int(match.group(1))/100)
    else:
        return 1

In [99]:
def get_review( url : str ,  airline : str ) -> list:
    
    '''
    get the review from one page
    
    '''
    
    # get the page
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    review_list= []
    allarticles = soup.findAll('article')
    articles = allarticles[0].findAll('article')


    for one in articles:
        record = {}
        record['airline'] = airline
        try : 
            overallRating = one.find('span',{'itemprop': 'ratingValue'}).text
            if str(overallRating) == 'None':
                overallRating = 0
            else:
                record['ratingValue'] = int(overallRating)

            record['author'] = one.find('span',{'itemprop': 'name'}).text.strip()

            tempdate = one.find('time',{'itemprop': 'datePublished'}).text
            tempdate = re.sub(r'(\d)(st|nd|rd|th)', r'\1', tempdate)
            date_time_obj = datetime.strptime(tempdate, '%d %B %Y')
            record['datePublished'] = date_time_obj

            tmptext = one.find('div',{'class': 'text_content'}).text
            text = tmptext.split('|')
            if len(text)>1:
                textindex=1
                verifyindex=0
            else:
                textindex=0
                verifyindex=-1
            record['reviewText'] = text[textindex].strip()

            if verifyindex!=-1:
                if 'Not' in tmptext.split('|')[verifyindex]:
                    record['verified'] = False
                else:
                    record['verified'] = True
            else:
                record['verified'] = False

            ## 其他細項
            for element in one.find_all('tr'):

                rating_element = element.find_all('td')[0]['class'][1]
                if not (element.find_all('td')[1].text).isnumeric():
                    rating_value = (element.find_all('td')[1]).text
                    record[rating_element] = rating_value.strip()
                else:
                    stars = len(element.find_all('span',{'class': 'star fill'}))
                    record[rating_element] = int(stars)

            review_list.append(record)
        except:
            pass

    return review_list
    

# Write to CSV

In [46]:
review_url_list = get_airlines()

In [101]:
# 匯出成csv
for idx , url in enumerate(review_url_list):
    
    total_review_list=[]

    # get airline name
    regex = re.compile(r'airline-reviews/(\S*)/page/1/\?sortby=post_date%3ADesc&pagesize=100')
    match = regex.search(url)
    airline = match.group(1)
    
    
    if idx%10==0:
        print(f'Now progress : {idx}')
        print(f'Now airline : {airline}')

    # get pages
    pages = get_pages(url)
    
    # get reviews
    str_ = 'https://www.airlinequality.com/airline-reviews/'
    for page in range(1,pages+1):
        tmp_url = str_+airline+'/page/'+str(page)+'/?sortby=post_date%3ADesc&pagesize=100'
        total_review_list += get_review(tmp_url , airline)
    
    # write to csv
    total_reviews_df = pd.DataFrame(total_review_list)
    total_reviews_df.to_csv('./Reviews/'+airline+'.csv', index=False)
    
        

        
  

Now progress : 0
Now airline : ab-aviation
Now progress : 10
Now airline : aerosur
Now progress : 20
Now airline : air-berlin
Now progress : 30
Now airline : air-costa
Now progress : 40
Now airline : air-italy
Now progress : 50
Now airline : air-mediterranee
Now progress : 60
Now airline : air-rarotonga
Now progress : 70
Now airline : airasia-philippines
Now progress : 80
Now airline : alaska-airlines
Now progress : 90
Now airline : andes-lineas-aereas
Now progress : 100
Now airline : atlasjet-airlines
Now progress : 110
Now airline : azul-linhas-aereas-brasileiras
Now progress : 120
Now airline : belavia
Now progress : 130
Now airline : blue1
Now progress : 140
Now airline : bulgaria-air
Now progress : 150
Now airline : caribbean-airlines
Now progress : 160
Now airline : china-southern-airlines
Now progress : 170
Now airline : corendon-airlines
Now progress : 180
Now airline : dniproavia
Now progress : 190
Now airline : egyptair
Now progress : 200
Now airline : europe-airpost
Now prog

# Check

In [102]:
import os 

dir_path = './Reviews/'
print(len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))

547
